<a href="https://colab.research.google.com/github/mahapara2411/Sentiment-Analysis/blob/master/TFIDF_cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np # linear algebra
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython.display import Image
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import files
uploaded = files.upload()

Saving hotel_review.csv to hotel_review.csv


In [5]:
import io
df = pd.read_csv(io.BytesIO(uploaded['hotel_review.csv']))

In [6]:
df.shape


(70, 6)

In [7]:
df.head()


,Hotel_id,Hotel,Review_id,Users,Reviews,Rating
0,g297628-d503249,Royal Orchid Central,r758572477,Afreen A,It was very memorable stay at Royal orchid cen...,5
1,g297628-d503249,Royal Orchid Central,r758007447,Srikumar D,The Things happen with the time remember for l...,5
2,g297628-d503249,Royal Orchid Central,r757841035,devendranakka,My. stay was very comfortable e delightful.Thi...,5
3,g297628-d503249,Royal Orchid Central,r757272486,Ashesha G Bhatta,I stayed here for 7 days & I was really happy ...,4
4,g297628-d503249,Royal Orchid Central,r757252794,Jaya,Had a very comfortable stay. Services of staff...,5


In [18]:
def clean_df(df_clean):
    remove_names = False # if True, assumes you have a nomes.txt file with common brazilian names in your current dir
    remove_usernames = False
    
    # Copy the original text for later metadata
    df_clean['original_text'] = df_clean['Reviews']

    # Lower case
    df_clean['Reviews'] = df_clean['Reviews'].apply(
        lambda x: " ".join(x.lower() for x in x.split()))

    # Remove usernames
    if remove_usernames:
        df_clean['Reviews'] = df_clean['Reviews'].str.replace(
            '@[^\s]+', "")

    # Remove links
    df_clean['Reviews'] = df_clean['Reviews'].str.replace(
        'https?:\/\/.*[\r\n]*', '')

    # Remove punctuation
    df_clean['Reviews'] = df_clean['Reviews'].str.replace(
        '[^\w\s]', '')
    # Remove stopwords
    from nltk.corpus import stopwords
    stop = stopwords.words('english')
    df_clean['Reviews'] = df_clean['Reviews'].apply(
        lambda x: " ".join(x for x in x.split() if x not in stop))

    

    # Remove numbers
    df_clean['Reviews'] = df_clean['Reviews'].str.replace(
        '\d+', '')
    # Remove words with 1-3 chars
    df_clean['Reviews'] = df_clean['Reviews'].str.replace(
        r'\b(\w{1,3})\b', '')

   
    
    return df_clean

In [19]:
df = clean_df(df)
df.head()

,Hotel_id,Hotel,Review_id,Users,Reviews,Rating,original_text
0,g297628-d503249,Royal Orchid Central,r758572477,Afreen A,memorable stay royal orchid central friendly s...,5,memorable stay royal orchid central friendly s...
1,g297628-d503249,Royal Orchid Central,r758007447,Srikumar D,things happen time remember long timethe trip ...,5,things happen time remember long timethe trip ...
2,g297628-d503249,Royal Orchid Central,r757841035,devendranakka,stay comfortable delightfulthis first visit bu...,5,stay comfortable delightfulthis first visit bu...
3,g297628-d503249,Royal Orchid Central,r757272486,Ashesha G Bhatta,stayed days really happy service provided wort...,4,stayed days really happy service provided wort...
4,g297628-d503249,Royal Orchid Central,r757252794,Jaya,comfortable stay services staff good ambience ...,5,comfortable stay services staff good ambience ...


In [42]:
Review = " ".join(review for review in df.Reviews)
wordcloud = WordCloud(
    width=3000,
    height=2000,
    background_color=('white').generate(Review)
plt.figure(figsize=(15,20))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

SyntaxError: ignored

In [27]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['Reviews'])
count_matrix

<70x1121 sparse matrix of type '<class 'numpy.int64'>'
	with 2362 stored elements in Compressed Sparse Row format>

In [26]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)

In [28]:
word_count

,word,count
0,hotel,71
1,staff,59
2,good,46
3,food,45
4,stay,43
...,...,...
1116,high,1
1117,highlight,1
1118,hima,1
1119,historical,1


In [30]:
df.head()


,Hotel_id,Hotel,Review_id,Users,Reviews,Rating,original_text
0,g297628-d503249,Royal Orchid Central,r758572477,Afreen A,memorable stay royal orchid central friendly s...,5,memorable stay royal orchid central friendly s...
1,g297628-d503249,Royal Orchid Central,r758007447,Srikumar D,things happen time remember long timethe trip ...,5,things happen time remember long timethe trip ...
2,g297628-d503249,Royal Orchid Central,r757841035,devendranakka,stay comfortable delightfulthis first visit bu...,5,stay comfortable delightfulthis first visit bu...
3,g297628-d503249,Royal Orchid Central,r757272486,Ashesha G Bhatta,stayed days really happy service provided wort...,4,stayed days really happy service provided wort...
4,g297628-d503249,Royal Orchid Central,r757252794,Jaya,comfortable stay services staff good ambience ...,5,comfortable stay services staff good ambience ...


In [31]:
tfidf_vectorizer = TfidfVectorizer()


In [32]:
X = tfidf_vectorizer.fit_transform(df['Reviews'])


In [33]:
print(tfidf_vectorizer.get_feature_names()[0:10])


['abhinandan', 'able', 'absolutely', 'access', 'accommodating', 'across', 'activities', 'actually', 'adil', 'advaith']


In [34]:
print(X.shape)


(70, 1121)


In [35]:
cosine_similarity(X[0:1], X).shape


(1, 70)

In [36]:
def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = cosine_similarity(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [38]:
df['Reviews'][10]


'visiting since found excellent service cosistently pavilion experience also great chief chef dhiren warm friendly informative rooms great service prompt'

In [39]:
for index, score in find_similar(X, 10):
       print("{} - {}".format(score, df['Reviews'][index]))

0.20183457210580866 - stayed many times still great hotel staff friendly courteous shubham housekeeping helpful took good care stay overall take care detail excellent service excellent rooms excellent staff rooms enough well decorated clean ipad control lights make calls many things great experience strongly recommend
0.1507664113436355 - pleasant memorable stay windsor suite thank mumtaz shefali sunil kumar excellent service food delicious pukht pavilion whole team friendly helpful
0.1252637777849627 - great stay staff warm friendly willing assist nothing much request breakfastrestaurant staff accommodating certainly consider staying hotel visiting bangalore western practice offered several attendants tips politely humility turned remarkable
0.1169976703901396 - awesome service terms customer experience house keeping best available point time call service desk available within minutesespecially tikaram epiphany house keeper responsive friendlythe food good finally want please visit gr